1. Importar librerías y definir funciones base

In [0]:
%pip install openpyxl
%pip install pyreadstat

In [0]:
%restart_python

## Carga desde archivo y construcción dicts

In [0]:
import os
import pandas as pd
import pyreadstat

directories = [
    "/Volumes/workspace/default/fallecidos_lesionados/",
    "/Volumes/workspace/default/hechos_transito/",
    "/Volumes/workspace/default/vehiculos_involucrados/"
]

for directory in directories:
    try:
        files = [f.name for f in dbutils.fs.ls(directory)]
        
        for file in files:
            lower_file = file.lower()
            file_path = os.path.join(directory, file)
            
            # XLSX
            if lower_file.endswith(".xlsx"):
                csv_path = file_path.replace(".xlsx", ".csv")
                try:
                    df = pd.read_excel(file_path)
                    df.to_csv(csv_path, index=False)
                    print(f"Convertido XLSX → CSV: {file_path} → {csv_path}")
                    
                    # Borrar original
                    dbutils.fs.rm(file_path)
                    print(f"Borrado XLSX: {file_path}")
                except Exception as e:
                    print(f"Error con XLSX {file_path}: {e}")
            
            # SAV
            elif lower_file.endswith(".sav"):
                csv_path = file_path.replace(".sav", ".csv")
                try:
                    df, meta = pyreadstat.read_sav(file_path)
                    df.to_csv(csv_path, index=False)
                    print(f"Convertido SAV → CSV: {file_path} → {csv_path}")
                    
                    # Borrar original
                    dbutils.fs.rm(file_path)
                    print(f"Borrado SAV: {file_path}")
                except Exception as e:
                    print(f"Error con SAV {file_path}: {e}")
                    
    except Exception as e:
        print(f"Error al procesar directorio {directory}: {e}")



Unir TODOS los CSV de una carpeta en un solo DataFrame (robusto a columnas distintas)

In [0]:
from pyspark.sql import functions as F
DIR_HECHOS = "/Volumes/workspace/default/hechos_transito"
DIR_VEHICULOS = "/Volumes/workspace/default/vehiculos_involucrados"
DIR_FALLECIDOS = "/Volumes/workspace/default/fallecidos_lesionados"

# Opción simple (si todos los CSV ya tienen mismas columnas):
hechos_raw = (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(f"{DIR_HECHOS}/*.csv")
    .withColumn("source_file", F.col("_metadata.file_path"))
)

display(hechos_raw.limit(5))
hechos_raw.printSchema()

vehiculos_raw = (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(f"{DIR_VEHICULOS}/*.csv")
    .withColumn("source_file", F.col("_metadata.file_path"))
)

display(vehiculos_raw.limit(5))
vehiculos_raw.printSchema()

fallecidos_raw = (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(f"{DIR_FALLECIDOS}/*.csv")
    .withColumn("source_file", F.col("_metadata.file_path"))
)

display(fallecidos_raw.limit(5))
fallecidos_raw.printSchema()

In [0]:
from pyspark.sql import functions as F, types as T

# 1) alias de columnas → nombre canónico
ALIASES = {
    "num_hecho":      ["num_hecho", "num", "num_correl", "num_corre", "n_um_corre", "num_correlativo", "núm_corre"],
    "anio":           ["anio", "ano_ocu", "año_ocu", "anio_ocu", "ano", "año"],
    "mes":            ["mes", "mes_ocu"],
    "dia":            ["dia", "día", "dia_ocu", "día_ocu"],
    "dia_sem":        ["dia_sem_ocu", "día_sem_ocu", "dia_sem", "día_sem"],
    "hora":           ["hora", "hora_ocu"],
    "g_hora":         ["g_hora", "g_hora_5"],
    "depto":          ["depto_ocu", "depto"],
    "mupio":          ["mupio_ocu", "muni_ocu", "municipio", "mupio"],
    "zona":           ["zona_ocu", "zona"],
    "area":           ["areag_ocu", "area_ocu", "area"],
    "tipo_accidente": ["tipo_eve", "tipo_evento", "tipo", "tipo_acc", "tipo_accidente"],
    "causa_acc":      ["causa_acc", "causa"],
    "sexo_pil":       ["sexo_pil", "sexo_piloto", "sexo", "sexo_per"],
    "edad_pil":       ["edad_pil", "edad", "edad_piloto", "edad_per"],
    "g_edad":         ["g_edad_2", "g_edad", "g_edad_80ymás", "g_edad_60ymás", "edad_quinquenales"],
    "mayor_menor":    ["mayor_menor"],
    "tipo_veh":       ["tipo_veh", "tipo_vehiculo"],
    "marca_veh":      ["marca_veh", "marca"],
    "color_veh":      ["color_veh", "color"],
    "modelo_veh":     ["modelo_veh", "modelo"],
    "g_modelo_veh":   ["g_modelo_veh"],
    "estado_pil":     ["estado_pil", "estado_piloto", "estado", "estado_con", "fall_les"],
    "intencionalidad":["int_o_noint"],
    "source_file":    ["source_file", "_source_file"]
}

def first_present(colnames, candidates):
    cols_lower = {c.lower(): c for c in colnames}
    for cand in candidates:
        if cand.lower() in cols_lower:
            return cols_lower[cand.lower()]
    return None

def canonize(df):
    # construir mapeo existente -> canon
    mapping = {}
    for canon, cands in ALIASES.items():
        src = first_present(df.columns, cands)
        if src and src != canon:
            mapping[src] = canon

    # aplicar renombres
    for src, dst in mapping.items():
        df = df.withColumnRenamed(src, dst)

    return df

hechos_std = canonize(hechos_raw)
fallecidos_std = canonize(fallecidos_raw)
vehiculos_std = canonize(vehiculos_raw)


In [0]:
# Si no hay 'anio', lo extraemos del nombre del archivo
if "anio" not in hechos_std.columns and "source_file" in hechos_std.columns:
    hechos_std = hechos_std.withColumn(
        "anio", F.regexp_extract("source_file", r"(20\d{2})", 1).cast("int")
    )

if "anio" not in fallecidos_std.columns and "source_file" in fallecidos_std.columns:
    fallecidos_std = fallecidos_std.withColumn(
        "anio", F.regexp_extract("source_file", r"(20\d{2})", 1).cast("int")
    )

if "anio" not in vehiculos_std.columns and "source_file" in vehiculos_std.columns:
    vehiculos_std = vehiculos_std.withColumn(
        "anio", F.regexp_extract("source_file", r"(20\d{2})", 1).cast("int")
    )

In [0]:
from pyspark.sql import functions as F, types as T

INT_COLS = ["anio","mes","dia","dia_sem","hora","g_hora","depto","mupio","zona","area",
            "tipo_accidente","causa_acc","sexo_pil","edad_pil","g_edad","mayor_menor",
            "tipo_veh","marca_veh","color_veh","modelo_veh","g_modelo_veh","estado_pil"]

# (1) Normaliza a string para inspección y limpia tokens comunes no-numéricos
TOKENS_NULOS = ["Ignorada","IGNORADA","NA","N/A","S/D","SD","Sin dato","SIN DATO","", " "]

dfs = [hechos_std, fallecidos_std, vehiculos_std]

for df in dfs:
    for c in INT_COLS:
        if c in df.columns:
            # todo como string primero
            s = F.col(c).cast("string")
            # convierte a NULL si coincide con tokens nulos
            s = F.when(F.lower(F.trim(s)).isin([t.lower() for t in TOKENS_NULOS]), None).otherwise(s)
            # deja pasar solo enteros (± dígitos); si no, NULL
            s = F.when(F.trim(s).rlike(r"^-?\d+$"), s).otherwise(None)
            # castea a int al final
            df = df.withColumn(c, s.cast(T.IntegerType()))

    # (2) regla específica: 9999 en modelo_veh -> NULL
    if "modelo_veh" in df.columns:
        df = df.withColumn("modelo_veh", F.when(F.col("modelo_veh")==9999, None).otherwise(F.col("modelo_veh")))


In [0]:
OUT_SILVER_HECHOS = "/Volumes/workspace/default/hechos_transito/silver"
OUT_SILVER_VEHICULOS = "/Volumes/workspace/default/vehiculos_involucrados/silver"
OUT_SILVER_FALLECIDOS = "/Volumes/workspace/default/fallecidos_lesionados/silver"

dfs = [hechos_std, fallecidos_std, vehiculos_std]
OUT_SILVERS = [OUT_SILVER_HECHOS, OUT_SILVER_VEHICULOS, OUT_SILVER_FALLECIDOS]

for i in range(3):
    df = dfs[i]
    out_silver = OUT_SILVERS[i]
    (df
    .coalesce(1)
    .write.mode("overwrite")
    .parquet(OUT_SILVER_HECHOS))

    print(f"{df} estandarizados →", out_silver)


Helpers reutilizables

### Preguntas a responder
1. Contar registros por tabla (long)

## #1 – Conteos, .show(), describe y summary (por tabla)

In [0]:
def ae_conteos_describe_summary(nombre: str, sdfs: dict, n_show: int = 5):
    print(f"\n===== {nombre.upper()} :: Conteos por hoja =====")
    total_registros = 0
    for key, sdf in sdfs.items():
        c = sdf.count()
        total_registros += c
        print(f"{key:20s} -> {c:6d} registros")
    print(f"TOTAL {nombre}: {total_registros}")

    # Muestra de una hoja representativa (la primera)
    if sdfs:
        first_key = list(sdfs.keys())[0]
        print(f"\n--- Ejemplo .show() :: {first_key} ---")
        sdfs[first_key].show(n_show, truncate=False)

        # describe/summary de columnas numéricas
        num_cols = [f.name for f in sdfs[first_key].schema.fields if isinstance(f.dataType, NumericType)]
        if num_cols:
            print(f"\n--- describe(numéricas) :: {first_key} ---")
            sdfs[first_key].select(*num_cols).describe().show(truncate=False)

            print(f"\n--- summary(numéricas) :: {first_key} ---")
            sdfs[first_key].select(*num_cols).summary("count","mean","stddev","min","25%","50%","75%","max").show(truncate=False)
        else:
            print(f"\n--- {first_key}: no hay columnas numéricas detectadas ---")

ae_conteos_describe_summary("hechos", hechos_sdfs)
ae_conteos_describe_summary("vehiculos", vehiculos_sdfs)
ae_conteos_describe_summary("lesionados", lesionados_sdfs)
ae_conteos_describe_summary("fallecidos", fallecidos_sdfs)

## #2 – Años disponibles por tabla y validación

In [0]:
EXPECTED_YEARS = set(range(2020, 2025))  # 2020..2024
YEAR_RX = re.compile(r'(?<!\d)(20\d{2})(?:\.0)?(?!\d)')

def extract_year_from_col(colname: str) -> int | None:
    m = YEAR_RX.search(str(colname))
    if not m: return None
    try:
        return int(m.group(1))
    except:
        return None

def detect_years_in_pdf(df: pd.DataFrame) -> set[int]:
    years = set()
    for c in df.columns:
        y = extract_year_from_col(c)
        if y: years.add(y)
    return years

def report_years_dict(dict_pd: dict, titulo: str):
    print(f"\n===== Verificación de años: {titulo} =====")
    all_years = set()
    for key, df in dict_pd.items():
        found = detect_years_in_pdf(df)
        all_years |= found
        missing = sorted(EXPECTED_YEARS - found)
        outside = sorted(y for y in found if y not in EXPECTED_YEARS)
        print(f"[{key}] encontrados: {sorted(found) if found else '—'} | faltantes vs 2020–2024: {missing if missing else 'ninguno'} | fuera de rango: {outside if outside else 'ninguno'}")
    print(f"\nAños agregados (union) en {titulo}: {sorted(all_years) if all_years else '—'}")
    return all_years

years_hechos     = report_years_dict(hechos_dfs, "hechos")
years_vehiculos  = report_years_dict(vehiculos_dfs, "vehiculos")
years_lesionados = report_years_dict(lesionados_dfs, "lesionados")
years_fallecidos = report_years_dict(fallecidos_dfs, "fallecidos")

print("\n¿Coinciden los conjuntos de años (intersección)?")
intersection_all = years_hechos & years_vehiculos & years_lesionados & years_fallecidos
print("Intersección común:", sorted(intersection_all) if intersection_all else "—")

### #3 – Valores distintos de 'tipo de accidente'   (buscamos columnas candidatas por nombre aproximado)

In [0]:
# === Accidente (largo + ancho) → tidy, robusto ===
import re, unicodedata
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, DoubleType, StructType, StructField

# --- normalizadores ---
def _norm_text(s):
    if s is None: return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")  # quita acentos
    s = re.sub(r"\s+", " ", s)
    return s

def _norm_colname(s):
    if s is None: return ""
    t = str(s).strip().lower()
    t = (t.replace("á","a").replace("é","e").replace("í","i")
           .replace("ó","o").replace("ú","u").replace("ñ","n"))
    t = re.sub(r"[\s\-]+", "_", t)
    return t

NOTE_PAT = re.compile(r"(fuente|nota|serie|el mes que no aparece|no se presentaron|sin datos)", re.IGNORECASE)

def _drop_noise_rows(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    # descarta filas de notas en cualquier columna
    mask_any = df.apply(lambda r: r.astype(str).str.contains(NOTE_PAT, na=False).any(), axis=1)
    return df.loc[~mask_any].copy()

# --- catálogo canónico y mapping ---
CANON = ["colision","Atropello","Derrape","Choque","Vuelco","Embarrancó","Encunetó","Caída","Ignorado"]
MAP_NORM_TO_CANON = {
    "colision":"colision","colisiones":"colision","colision multiple":"colision",
    "atropello":"Atropello","atropellos":"Atropello",
    "derrape":"Derrape","derrapes":"Derrape",
    "choque":"Choque","choques":"Choque",
    "vuelco":"Vuelco","vuelcos":"Vuelco",
    "embarranco":"Embarrancó","embarranco multiple":"Embarrancó",
    "encuneto":"Encunetó","encunetamiento":"Encunetó",
    "caida":"Caída","caidas":"Caída",
    "ignorado":"Ignorado","desconocido":"Ignorado","no especificado":"Ignorado","sin dato":"Ignorado","na":"Ignorado","n/a":"Ignorado",
}

@F.udf(returnType=StringType())
def canon_val_udf(val):
    if val is None: return None
    n = _norm_text(val)
    if not n: return None
    return MAP_NORM_TO_CANON.get(n, "Ignorado")

def _find_long_tipo_col(df: pd.DataFrame) -> str | None:
    # busca “tipo_de_accidente” (normalizado) o abreviado “tipo_de_accid”
    for c in df.columns:
        nc = _norm_colname(c)
        if nc == "tipo_de_accidente" or nc.startswith("tipo_de_accid"):
            return c
    return None

def _find_wide_cols(df: pd.DataFrame):
    """Devuelve (cols, rename) para columnas de accidentes en formato ANCHO."""
    cols, ren = [], {}
    for c in df.columns:
        n = _norm_text(c)
        if n in MAP_NORM_TO_CANON:
            canon = MAP_NORM_TO_CANON[n]
            # acepta solo si parece conteo (al menos un número en la col)
            s = pd.to_numeric(df[c], errors="coerce")
            if s.notna().any():
                cols.append(c)
                ren[c] = canon
    return cols, ren

def _build_tidy_from_long(df: pd.DataFrame, sheet_name: str):
    # espera columna "tipo de accidente" + columnas de conteos (años/meses)
    tipo_col = _find_long_tipo_col(df)
    if not tipo_col: 
        return None  # no es formato largo
    work = _drop_noise_rows(df)
    # columnas numéricas candidatas (conteos)
    num_cols = []
    for c in work.columns:
        if c == tipo_col: continue
        if pd.to_numeric(work[c], errors="coerce").notna().mean() > 0.3:
            num_cols.append(c)
    if not num_cols: 
        return None
    # sumar por tipo (excluye filas 'total' en cualquier col de texto)
    mask_total = work.apply(lambda r: r.astype(str).str.strip().str.lower().eq('total').any(), axis=1)
    work = work.loc[~mask_total].copy()
    work["__tipo__"] = work[tipo_col].map(lambda x: MAP_NORM_TO_CANON.get(_norm_text(x), "Ignorado"))
    sums = work[num_cols].apply(pd.to_numeric, errors="coerce").fillna(0).groupby(work["__tipo__"]).sum().sum(axis=1)
    tidy_pd = sums.reset_index().rename(columns={"__tipo__":"accidente",0:"count"})
    tidy_pd["sheet"] = sheet_name
    return tidy_pd[["sheet","accidente","count"]]

def _build_tidy_from_wide(df: pd.DataFrame, sheet_name: str):
    cols, ren = _find_wide_cols(df)
    if not cols: 
        return None
    work = _drop_noise_rows(df)
    pdf = work[cols].rename(columns=ren).copy()
    for c in pdf.columns:
        pdf[c] = pd.to_numeric(pdf[c], errors="coerce")
    # Si hay filas “Total” en alguna col de texto ajena, igual sumar todo (salvo si quieres excluir explícitamente)
    sums = {canon: float(pdf.get(canon, pd.Series(dtype=float)).fillna(0).sum()) for canon in CANON}
    tidy_pd = pd.DataFrame({"sheet": [sheet_name]*len(CANON),
                            "accidente": CANON,
                            "count": [sums[k] for k in CANON]})
    return tidy_pd

def accidentes_tidy_from_dict(dict_pd: dict, titulo: str):
    """Devuelve un Spark DF tidy con columnas: sheet, accidente, count, combinando hojas largo/ancho."""
    rows = []
    for key, df in dict_pd.items():
        t_long = _build_tidy_from_long(df, key)
        if t_long is not None:
            rows.append(t_long)
            continue
        t_wide = _build_tidy_from_wide(df, key)
        if t_wide is not None:
            rows.append(t_wide)
    if not rows:
        print(f"[{titulo}] No encontré info de accidentes en este conjunto.")
        return spark.createDataFrame([], schema=StructType([
            StructField("sheet", StringType(), True),
            StructField("accidente", StringType(), True),
            StructField("count", DoubleType(), True),
        ]))
    tidy_all = pd.concat(rows, ignore_index=True)
    sdf = spark.createDataFrame(tidy_all.astype({"sheet":"string","accidente":"string","count":"float"}))
    print(f"[{titulo}] {sdf.count()} filas tidy (sheet, accidente, count)")
    return sdf

# === USO ===
acc_hechos_sdf     = accidentes_tidy_from_dict(hechos_dfs, "hechos")
acc_vehiculos_sdf  = accidentes_tidy_from_dict(vehiculos_dfs, "vehiculos")
acc_lesionados_sdf = accidentes_tidy_from_dict(lesionados_dfs, "lesionados")
acc_fallecidos_sdf = accidentes_tidy_from_dict(fallecidos_dfs, "fallecidos")

# Ejemplos de consulta:
print("\nTop 10 (hechos) por count:")
acc_hechos_sdf.groupBy("accidente").agg(F.sum("count").alias("total")).orderBy(F.desc("total")).show(10, truncate=False)

print("\nHojas que sólo tuvieron 'Ignorado' (vehículos):")
(acc_vehiculos_sdf
 .groupBy("sheet")
 .pivot("accidente")
 .agg(F.sum("count"))
 .where(F.col("Ignorado").isNotNull() &
        sum(F.col(c).isNull() | (F.col(c) == 0) for c in CANON if c != "Ignorado") == (len(CANON)-1))
 .select("sheet","Ignorado")
 .show(truncate=False))


### #4 – # de departamentos únicos por base(detecta columna 'departamento' aproximada)

In [0]:
def count_departamentos_unique(dict_sdfs: dict, titulo: str,
                               dep_cands=("departamento","departamentos","depto","dept","depart","depar")):
    print(f"\n===== Departamentos únicos :: {titulo} =====")
    deptos = set()
    cols_encontradas = 0

    # Aux: si no tienes ya definida find_first_column, deja esto aquí
    def find_first_column(candidates: list[str], columns: list[str]) -> str | None:
        for col in columns:
            low = col.lower()
            for cand in candidates:
                if cand in low:  # match por substring
                    return col
        return None

    for key, sdf in dict_sdfs.items():
        col = find_first_column(list(dep_cands), sdf.columns)
        if not col:
            continue
        cols_encontradas += 1

        vals = (sdf
                .select(F.col(col).cast("string").alias("departamento"))
                .where(F.col("departamento").isNotNull() & (F.col("departamento") != ""))
                .distinct()
                .toPandas()["departamento"])

        for v in vals:
            if v is not None:
                deptos.add(str(v).strip())

    print(f"Total únicos (unión de hojas): {len(deptos)}")
    print(f"Columnas 'departamento' detectadas en {cols_encontradas} hojas")
    if deptos:
        print("Ejemplos:", sorted(list(deptos))[:15])
count_departamentos_unique(hechos_sdfs, "hechos")
count_departamentos_unique(vehiculos_sdfs, "vehiculos")
count_departamentos_unique(lesionados_sdfs, "lesionados")
count_departamentos_unique(fallecidos_sdfs, "fallecidos")


5. ¿Cuál es el total de accidentes por año y departamento?

6. ¿Qué día de la semana registra más accidentes en 2024?